In [1]:
print("hi")

hi


In [2]:
# Lab 3: Stemming and Lemmatization for Text Preprocessing
# Import required libraries
import pandas as pd
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\D
[nltk_data]     Navya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\D
[nltk_data]     Navya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\D
[nltk_data]     Navya\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt_tab to C:\Users\D
[nltk_data]     Navya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\D
[nltk_data]     Navya\AppData\Roaming\nltk_data...


True

In [5]:
# 1. Load the dataset into a Pandas DataFrame and extract the text and label column

# Load the cleaned dataset
df = pd.read_csv('cleaned_data.csv', index_col=0)

# Extract text and label columns
texts = df['review_text'].tolist()
labels = df['sentiment'].tolist()

print(f"Total documents loaded: {len(texts)}")
print(f"Total labels loaded: {len(labels)}")
print(f"\nSample document:")
print(f"Text: {texts[0]}")
print(f"Label: {labels[0]}")

Total documents loaded: 100
Total labels loaded: 100

Sample document:
Text: oooh, sunshine! A patch of sunshine! And it will be gone by the time I leave work and replaced with rain.  /vent
Label: neutral


In [7]:
# 2. Perform tokenization on all documents and store the tokens

# Tokenize all documents
tokenized_docs = []
for text in texts:
    tokens = word_tokenize(text)
    tokenized_docs.append(tokens)

print(f"Total documents tokenized: {len(tokenized_docs)}")
print(f"\nExample - First document:")
print(f"Original: {texts[0][:100]}...")
print(f"Tokens: {tokenized_docs[0][:15]}")
print(f"Total tokens in first doc: {len(tokenized_docs[0])}")

Total documents tokenized: 100

Example - First document:
Original: oooh, sunshine! A patch of sunshine! And it will be gone by the time I leave work and replaced with ...
Tokens: ['oooh', ',', 'sunshine', '!', 'A', 'patch', 'of', 'sunshine', '!', 'And', 'it', 'will', 'be', 'gone', 'by']
Total tokens in first doc: 26


In [8]:
# 3. Apply case folding by converting all tokens to lowercase

# Convert all tokens to lowercase
lowercase_docs = []
for tokens in tokenized_docs:
    lowercase_tokens = [token.lower() for token in tokens]
    lowercase_docs.append(lowercase_tokens)

print(f"Case folding applied to all {len(lowercase_docs)} documents")
print(f"\nExample - First document after case folding:")
print(f"Before: {tokenized_docs[0][:10]}")
print(f"After: {lowercase_docs[0][:10]}")

Case folding applied to all 100 documents

Example - First document after case folding:
Before: ['oooh', ',', 'sunshine', '!', 'A', 'patch', 'of', 'sunshine', '!', 'And']
After: ['oooh', ',', 'sunshine', '!', 'a', 'patch', 'of', 'sunshine', '!', 'and']


In [ ]:
# 4. Remove stop-words from the tokenized documents

# Load English stop-words
stop_words = set(stopwords.words('english'))

# Remove stop-words and non-alphabetic tokens
filtered_docs = []
for tokens in lowercase_docs:
    filtered_tokens = [token for token in tokens 
                      if token not in stop_words and token.isalpha()]
    filtered_docs.append(filtered_tokens)

print(f"Stop-words loaded: {len(stop_words)}")
print(f"\nExample - First document after stop-word removal:")
print(f"Before (lowercase): {lowercase_docs[0][:25]}")
print(f"After (filtered): {filtered_docs[0][:25]}")
print(f"Tokens before: {len(lowercase_docs[0])}, Tokens after: {len(filtered_docs[0])}")

Stop-words loaded: 198

Example - First document after stop-word removal:
Before (lowercase): ['oooh', ',', 'sunshine', '!', 'a', 'patch', 'of', 'sunshine', '!', 'and', 'it', 'will', 'be', 'gone', 'by', 'the', 'time', 'i', 'leave', 'work', 'and', 'replaced', 'with', 'rain', '.']
After (filtered): ['oooh', 'sunshine', 'patch', 'sunshine', 'gone', 'time', 'leave', 'work', 'replaced', 'rain']
Tokens before: 26, Tokens after: 10


In [11]:
# 5. Apply stemming to the filtered tokens

# Initialize Porter Stemmer
stemmer = PorterStemmer()

# Apply stemming to all filtered documents
stemmed_docs = []
for tokens in filtered_docs:
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    stemmed_docs.append(stemmed_tokens)

print(f"Stemming applied to all {len(stemmed_docs)} documents")
print(f"\nExample - Stemming transformations:")
sample_tokens = filtered_docs[0][:10]
sample_stemmed = stemmed_docs[0][:10]
for original, stemmed in zip(sample_tokens, sample_stemmed):
    if original != stemmed:
        print(f"  {original:25} -> {stemmed}")

Stemming applied to all 100 documents

Example - Stemming transformations:
  sunshine                  -> sunshin
  sunshine                  -> sunshin
  leave                     -> leav
  replaced                  -> replac


In [12]:
# 6. Create a new column with final preprocessed text after stemming

# Join stemmed tokens back into text
df['stemmed_text'] = [' '.join(tokens) for tokens in stemmed_docs]

print(f"Stemmed text column created")
print(f"\nExample - First document:")
print(f"Original: {texts[0][:100]}...")
print(f"Stemmed: {df['stemmed_text'].iloc[0][:100]}...")

Stemmed text column created

Example - First document:
Original: oooh, sunshine! A patch of sunshine! And it will be gone by the time I leave work and replaced with ...
Stemmed: oooh sunshin patch sunshin gone time leav work replac rain...


In [13]:
# 7. Construct the Bag-of-Words vocabulary using stemmed text


# Collect all stemmed tokens
all_stemmed_tokens = []
for tokens in stemmed_docs:
    all_stemmed_tokens.extend(tokens)

# Create vocabulary (unique tokens)
stemmed_vocabulary = sorted(list(set(all_stemmed_tokens)))

print(f"Total stemmed tokens (with repetition): {len(all_stemmed_tokens)}")
print(f"Stemmed vocabulary size: {len(stemmed_vocabulary)}")
print(f"\nSample words from stemmed vocabulary: {stemmed_vocabulary[:20]}")

Total stemmed tokens (with repetition): 701
Stemmed vocabulary size: 487

Sample words from stemmed vocabulary: ['abil', 'absout', 'accept', 'account', 'act', 'ad', 'adapt', 'adob', 'age', 'agenc', 'ago', 'agre', 'ah', 'ahem', 'ahhh', 'alreadi', 'alright', 'also', 'alway', 'anim']


In [18]:
# 8. Determine size and top 10 most frequent words (stemmed)

# Count word frequencies
stemmed_word_freq = Counter(all_stemmed_tokens)
top_10_stemmed = stemmed_word_freq.most_common(10)

print(f"Stemming-based vocabulary size: {len(stemmed_vocabulary)}")
print(f"\nTop 10 most frequent words:")
for rank, (word, freq) in enumerate(top_10_stemmed, 1):
    print(f"  {rank}. {word} - {freq} occurrences")

Stemming-based vocabulary size: 487

Top 10 most frequent words:
  1. day - 8 occurrences
  2. go - 7 occurrences
  3. know - 7 occurrences
  4. work - 6 occurrences
  5. back - 6 occurrences
  6. miss - 6 occurrences
  7. im - 6 occurrences
  8. get - 5 occurrences
  9. like - 5 occurrences
  10. happi - 5 occurrences


In [19]:
# 9. Generate Document-Term Matrix (DTM) for stemmed text

# Create word to index mapping
stemmed_word_to_idx = {word: idx for idx, word in enumerate(stemmed_vocabulary)}

# Create DTM
stemmed_dtm = []
for tokens in stemmed_docs:
    vector = [0] * len(stemmed_vocabulary)
    for token in tokens:
        if token in stemmed_word_to_idx:
            vector[stemmed_word_to_idx[token]] += 1
    stemmed_dtm.append(vector)

stemmed_dtm = np.array(stemmed_dtm)

print(f"Stemmed DTM shape: {stemmed_dtm.shape}")
print(f"(Rows = {stemmed_dtm.shape[0]} documents, Columns = {stemmed_dtm.shape[1]} vocabulary)")
print(f"\nFirst document vector (first 20 dimensions): {stemmed_dtm[0][:20]}")
print(f"Non-zero features in first document: {np.count_nonzero(stemmed_dtm[0])}")

Stemmed DTM shape: (100, 487)
(Rows = 100 documents, Columns = 487 vocabulary)

First document vector (first 20 dimensions): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Non-zero features in first document: 9


In [22]:
# 10. Apply lemmatization to the filtered tokens

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to all filtered documents
lemmatized_docs = []
for tokens in filtered_docs:
    # Lemmatize as nouns first, then as verbs for better results
    lemmatized_tokens = [lemmatizer.lemmatize(lemmatizer.lemmatize(word), pos='v') 
                        for word in tokens]
    lemmatized_docs.append(lemmatized_tokens)

print(f"Lemmatization applied to all {len(lemmatized_docs)} documents")
print(f"\nExample - Lemmatization transformations:")
sample_tokens = filtered_docs[0][:10]
sample_lemmatized = lemmatized_docs[0][:10]
for original, lemmatized in zip(sample_tokens, sample_lemmatized):
    if original != lemmatized:
        print(f"  {original} -> {lemmatized}")

Lemmatization applied to all 100 documents

Example - Lemmatization transformations:
  gone -> go
  replaced -> replace


In [25]:
# 11. Create a new column with final preprocessed text after lemmatization

# Join lemmatized tokens back into text
df['lemmatized_text'] = [' '.join(tokens) for tokens in lemmatized_docs]

print(f"Lemmatized text column created")
print(f"\nExample - First document:")
print(f"Original: {texts[0:4][:100]}...")
print(f"Lemmatized: {df['lemmatized_text'].iloc[0:4][:100]}...")

Lemmatized text column created

Example - First document:
Original: ['oooh, sunshine! A patch of sunshine! And it will be gone by the time I leave work and replaced with rain.  /vent', 'skype call with billie but my webcam dont work', 'Just had a great study time followed by a delicious Japanese meal with Arty! Now, trying to get back into the study mood', 'FML not having a car is prohibiting finding a job']...
Lemmatized: 1    oooh sunshine patch sunshine go time leave wor...
2                   skype call billie webcam dont work
3    great study time follow delicious japanese mea...
4                            fml car prohibit find job
Name: lemmatized_text, dtype: object...


In [26]:
# 12. Construct the Bag-of-Words vocabulary using lemmatized text


# Collect all lemmatized tokens
all_lemmatized_tokens = []
for tokens in lemmatized_docs:
    all_lemmatized_tokens.extend(tokens)

# Create vocabulary (unique tokens)
lemmatized_vocabulary = sorted(list(set(all_lemmatized_tokens)))

print(f"Total lemmatized tokens (with repetition): {len(all_lemmatized_tokens)}")
print(f"Lemmatized vocabulary size: {len(lemmatized_vocabulary)}")
print(f"\nSample words from lemmatized vocabulary: {lemmatized_vocabulary[:20]}")

Total lemmatized tokens (with repetition): 701
Lemmatized vocabulary size: 479

Sample words from lemmatized vocabulary: ['ability', 'absoute', 'accept', 'account', 'act', 'ad', 'adapt', 'adobe', 'age', 'agency', 'ago', 'agree', 'ah', 'ahem', 'ahhh', 'already', 'alright', 'also', 'always', 'animal']


In [28]:
# 13. Determine size and top 10 most frequent words (lemmatized)

# Count word frequencies
lemmatized_word_freq = Counter(all_lemmatized_tokens)
top_10_lemmatized = lemmatized_word_freq.most_common(10)

print(f"Lemmatization-based vocabulary size: {len(lemmatized_vocabulary)}")
print(f"\nTop 10 most frequent words (lemmatized):")
for rank, (word, freq) in enumerate(top_10_lemmatized, 1):
    print(f"  {rank}. {word} - {freq} occurrences")

Lemmatization-based vocabulary size: 479

Top 10 most frequent words (lemmatized):
  1. get - 9 occurrences
  2. go - 8 occurrences
  3. day - 8 occurrences
  4. know - 8 occurrences
  5. work - 6 occurrences
  6. back - 6 occurrences
  7. miss - 6 occurrences
  8. make - 6 occurrences
  9. im - 6 occurrences
  10. like - 5 occurrences


In [29]:
# 14. Generate Document-Term Matrix (DTM) for lemmatized text

# Create word to index mapping
lemmatized_word_to_idx = {word: idx for idx, word in enumerate(lemmatized_vocabulary)}

# Create DTM
lemmatized_dtm = []
for tokens in lemmatized_docs:
    vector = [0] * len(lemmatized_vocabulary)
    for token in tokens:
        if token in lemmatized_word_to_idx:
            vector[lemmatized_word_to_idx[token]] += 1
    lemmatized_dtm.append(vector)

lemmatized_dtm = np.array(lemmatized_dtm)

print(f"Lemmatized DTM shape: {lemmatized_dtm.shape}")
print(f"(Rows = {lemmatized_dtm.shape[0]} documents, Columns = {lemmatized_dtm.shape[1]} vocabulary)")
print(f"\nFirst document vector (first 20 dimensions): {lemmatized_dtm[0][:20]}")
print(f"Non-zero features in first document: {np.count_nonzero(lemmatized_dtm[0])}")

Lemmatized DTM shape: (100, 479)
(Rows = 100 documents, Columns = 479 vocabulary)

First document vector (first 20 dimensions): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Non-zero features in first document: 9


In [32]:
# 15. Select one document and display its BoW vector from both DTMs

doc_idx = 4

print(f"Selected Document (Index {doc_idx}):")
print(f"Original text: {texts[doc_idx]}")
print(f"\nStemmed text: {df['stemmed_text'].iloc[doc_idx]}")
print(f"Lemmatized text: {df['lemmatized_text'].iloc[doc_idx]}")

print(f"\nStemming-based BoW Vector")
print(f"Vector length: {len(stemmed_dtm[doc_idx])}")
print(f"Non-zero elements: {np.count_nonzero(stemmed_dtm[doc_idx])}")
print(f"Vector (first 30 dimensions): {stemmed_dtm[doc_idx][:30]}")

# Show non-zero features with their words
print(f"\nNon-zero features (word: count):")
for idx, count in enumerate(stemmed_dtm[doc_idx]):
    if count > 0:
        print(f"  {stemmed_vocabulary[idx]} : {count}")

print(f"\nLemmatization-based BoW Vector")
print(f"Vector length: {len(lemmatized_dtm[doc_idx])}")
print(f"Non-zero elements: {np.count_nonzero(lemmatized_dtm[doc_idx])}")
print(f"Vector (first 30 dimensions): {lemmatized_dtm[doc_idx][:30]}")

# Show non-zero features with their words
print(f"\nNon-zero features (word: count):")
for idx, count in enumerate(lemmatized_dtm[doc_idx]):
    if count > 0:
        print(f"  {lemmatized_vocabulary[idx]} : {count}")

Selected Document (Index 4):
Original text:   lol since I got twitter a little while ago, it seems like loads of people setting up an account

Stemmed text: lol sinc got twitter littl ago seem like load peopl set account
Lemmatized text: lol since get twitter little ago seem like load people set account

Stemming-based BoW Vector
Vector length: 487
Non-zero elements: 12
Vector (first 30 dimensions): [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Non-zero features (word: count):
  account : 1
  ago : 1
  got : 1
  like : 1
  littl : 1
  load : 1
  lol : 1
  peopl : 1
  seem : 1
  set : 1
  sinc : 1
  twitter : 1

Lemmatization-based BoW Vector
Vector length: 479
Non-zero elements: 12
Vector (first 30 dimensions): [0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Non-zero features (word: count):
  account : 1
  ago : 1
  get : 1
  like : 1
  little : 1
  load : 1
  lol : 1
  people : 1
  seem : 1
  set : 1
  since : 1
  twitter : 1


In [36]:
# 16. Identify words appearing in most/few documents and discuss significance

# Calculate document frequency for stemmed vocabulary
stemmed_doc_freq = {}
for word in stemmed_vocabulary:
    doc_count = 0
    for tokens in stemmed_docs:
        if word in tokens:
            doc_count += 1
    stemmed_doc_freq[word] = doc_count

# Calculate document frequency for lemmatized vocabulary
lemmatized_doc_freq = {}
for word in lemmatized_vocabulary:
    doc_count = 0
    for tokens in lemmatized_docs:
        if word in tokens:
            doc_count += 1
    lemmatized_doc_freq[word] = doc_count

# Find words appearing in most documents (stemmed)
sorted_stemmed = sorted(stemmed_doc_freq.items(), key=lambda x: x[1], reverse=True)
most_common_stemmed = sorted_stemmed[:10]
least_common_stemmed = [item for item in sorted_stemmed if item[1] == 1][:10]

# Find words appearing in most documents (lemmatized)
sorted_lemmatized = sorted(lemmatized_doc_freq.items(), key=lambda x: x[1], reverse=True)
most_common_lemmatized = sorted_lemmatized[:10]
least_common_lemmatized = [item for item in sorted_lemmatized if item[1] == 1][:10]

print("STEMMED VOCABULARY")
print(f"\nWords appearing in MOST documents (Top 10):")
for word, doc_count in most_common_stemmed:
    percentage = (doc_count / len(texts)) * 100
    print(f"  {word} appears in {doc_count} documents ({percentage}%)")

print(f"\nWords appearing in VERY FEW documents (appearing in only 1 document):")
print(f"Total such words: {len([item for item in sorted_stemmed if item[1] == 1])}")
print(f"Examples:")
for word, doc_count in least_common_stemmed:
    print(f"  {word} appears in {doc_count} document")

print("\nLEMMATIZED VOCABULARY")
print(f"\nWords appearing in MOST documents (Top 10):")
for word, doc_count in most_common_lemmatized:
    percentage = (doc_count / len(texts)) * 100
    print(f"  {word} appears in {doc_count} documents ({percentage}%)")

print(f"\nWords appearing in VERY FEW documents (appearing in only 1 document):")
print(f"Total such words: {len([item for item in sorted_lemmatized if item[1] == 1])}")
print(f"Examples:")
for word, doc_count in least_common_lemmatized:
    print(f"  {word} appears in {doc_count} document")

STEMMED VOCABULARY

Words appearing in MOST documents (Top 10):
  day appears in 8 documents (8.0%)
  know appears in 7 documents (7.000000000000001%)
  back appears in 6 documents (6.0%)
  go appears in 6 documents (6.0%)
  miss appears in 6 documents (6.0%)
  work appears in 6 documents (6.0%)
  get appears in 5 documents (5.0%)
  happi appears in 5 documents (5.0%)
  http appears in 5 documents (5.0%)
  im appears in 5 documents (5.0%)

Words appearing in VERY FEW documents (appearing in only 1 document):
Total such words: 369
Examples:
  abil appears in 1 document
  absout appears in 1 document
  accept appears in 1 document
  account appears in 1 document
  act appears in 1 document
  ad appears in 1 document
  adapt appears in 1 document
  adob appears in 1 document
  age appears in 1 document
  agenc appears in 1 document

LEMMATIZED VOCABULARY

Words appearing in MOST documents (Top 10):
  get appears in 9 documents (9.0%)
  day appears in 8 documents (8.0%)
  know appears in 8

In [38]:
print("\nhigh frequency words (appearing in most documents):")
print("stemmed: 'day' (8 docs), 'go' (7 docs), 'know' (7 docs)")
print("lemmatized: 'get' (9 docs), 'day' (8 docs), 'know' (8 docs)")
print("\nwhat this means:")
print("- even top words appear in less than 10% of documents")
print("- these are general action verbs: get, go, know, work, miss")
print("- they dont carry specific sentiment (neutral words)")
print("- appear in positive, negative, and neutral reviews equally")
print("- not very useful for distinguishing between sentiment classes")
print("- better for identifying general topics rather than sentiment")


high frequency words (appearing in most documents):
stemmed: 'day' (8 docs), 'go' (7 docs), 'know' (7 docs)
lemmatized: 'get' (9 docs), 'day' (8 docs), 'know' (8 docs)

what this means:
- even top words appear in less than 10% of documents
- these are general action verbs: get, go, know, work, miss
- they dont carry specific sentiment (neutral words)
- appear in positive, negative, and neutral reviews equally
- not very useful for distinguishing between sentiment classes
- better for identifying general topics rather than sentiment


In [39]:
print("\nfor sentiment classification:")
print("ideal features would be:")
print("- words appearing in 20-50% of documents")
print("- sentiment-bearing words: love, hate, great, terrible")
print("- currently missing because dataset is small (100 docs)")
print("- most words are too rare to be reliable features")
print("- model might struggle due to high sparsity")


for sentiment classification:
ideal features would be:
- words appearing in 20-50% of documents
- sentiment-bearing words: love, hate, great, terrible
- currently missing because dataset is small (100 docs)
- most words are too rare to be reliable features
- model might struggle due to high sparsity


In [40]:
print("the high proportion of rare words (75%) indicates:")
print("1. very diverse vocabulary across documents")
print("2. small dataset with limited word repetition")
print("3. need for more data or different feature engineering")
print("4. tf-idf might help reduce impact of rare words")
print("5. lemmatization slightly better than stemming here")

the high proportion of rare words (75%) indicates:
1. very diverse vocabulary across documents
2. small dataset with limited word repetition
3. need for more data or different feature engineering
4. tf-idf might help reduce impact of rare words
5. lemmatization slightly better than stemming here
